<a href="https://colab.research.google.com/github/Natdata/Dodatki---wsp-lni-u-ytkownicy/blob/main/Dodatki_wsp%C3%B3lni_widzowie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Obliczanie userów dodatku i oryginału oraz konwersji

In [ ]:
import pandas as pd
import requests
from pandas import json_normalize

# Function to fetch leaflet data from API
def fetch_leaflet_data(leaflet_id, start, end, headers):
    final_df = pd.DataFrame()
    data = {
        "event_name": "leaflet_enter",
        "event_properties": [{"name": "leaflet_id", "operator": "equals", "value": leaflet_id}],
        "from": start,
        "to": end
    }

    try:
        response = requests.post('https://eu1.api.clevertap.com/1/events.json', headers=headers, json=data)
        response.raise_for_status()  # Ensure we raise an exception for bad status codes
        response_json = response.json()
        req_id = response_json.get('cursor')

        while req_id:
            print(f"Fetching data for cursor: {req_id}")  # Logging the cursor to track progress
            params = (('cursor', req_id),)
            response1 = requests.get(f'https://eu1.api.clevertap.com/1/events.json?cursor={req_id}', headers=headers, params=params)
            response1.raise_for_status()  # Ensure we raise an exception for bad status codes
            response_data = response1.json()
            records = response_data.get('records', [])
            response_df = json_normalize(records)

            if 'ts' in response_df.columns:
                response_df = response_df[['ts', 'event_props.leaflet_id', 'profile.identity']]
                response_df.rename(columns={'event_props.leaflet_id': 'leaflet_id', 'profile.identity': 'user_id'}, inplace=True)
                response_df['ts'] = response_df['ts'].astype(str).apply(lambda x: x[:8])
                final_df = pd.concat([final_df, response_df], ignore_index=True)

            req_id = response_data.get('next_cursor')
            if not req_id:  # If no next cursor, break the loop
                break

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

    return final_df

# API headers
headers = {
    'X-CleverTap-Account-Id': 'aaa',
    'X-CleverTap-Passcode': 'bbb',
    'Content-Type': 'application/json',
}

# Date range for the data
start = '2024-06-14'
end = '2024-06-19'

# Leaflet IDs
  leaflet_ids = ['367508', '365597']  # 1ID - dodatek, 2ID Oryginał

# Convert dates to integers
start = int(start.replace('-', ''))
end = int(end.replace('-', ''))

# Fetch data for both leaflets
leaflet_data_1 = fetch_leaflet_data(leaflet_ids[0], start, end, headers)
leaflet_data_2 = fetch_leaflet_data(leaflet_ids[1], start, end, headers)

# Combine the data into a single DataFrame
conversion = pd.concat([leaflet_data_1, leaflet_data_2], ignore_index=True)

# Save the combined data to a CSV file
#conversion.to_csv('combined_leaflet_data.csv', index=False)

# Unique users for each leaflet
unique_users_leaflet_1 = set(leaflet_data_1['user_id'])
unique_users_leaflet_2 = set(leaflet_data_2['user_id'])

# Users only in leaflet 1
users_only_in_leaflet_1 = unique_users_leaflet_1 - unique_users_leaflet_2

# Users who entered leaflet 1 first, then leaflet 2
leaflet_data_1_sorted = leaflet_data_1.sort_values(by='ts')
leaflet_data_2_sorted = leaflet_data_2.sort_values(by='ts')

# Create a DataFrame with the first entry of each user in each leaflet
first_entry_leaflet_1 = leaflet_data_1_sorted.groupby('user_id').first().reset_index()
first_entry_leaflet_2 = leaflet_data_2_sorted.groupby('user_id').first().reset_index()

# Merge the DataFrames to find users who entered both leaflets
merged_entries = pd.merge(first_entry_leaflet_1, first_entry_leaflet_2, on='user_id', suffixes=('_leaflet_1', '_leaflet_2'))

# Find users who entered leaflet 1 first, then leaflet 2
unique_first_leaflet_1_then_leaflet_2 = merged_entries[merged_entries['ts_leaflet_1'] < merged_entries['ts_leaflet_2']]['user_id'].unique()

# Display results
print("Data has been saved to 'combined_leaflet_data.csv'.")

print("1. Liczba unikatowych użytkowników, którzy weszli w dodatek, ale nie weszli w oryginał:")
print(len(users_only_in_leaflet_1))

print("\n2. Liczba unikatowych użytkowników dodatku:")
print(len(unique_users_leaflet_1))

print("\n3. Liczba unikatowych użytkowników oryginału:")
print(len(unique_users_leaflet_2))

print("\n4. Liczba unikatowych użytkowników, którzy weszli najpierw w dodatek, a następnie w oryginał:")
print(len(unique_first_leaflet_1_then_leaflet_2))


Fetching data for cursor: ZyZjfgABAQdvb2t5Kz8NegUCAAZmb2R4bmpkfE4BBgZmaGN%2BYWlrewQHTQZmbS57K2oue04ATQQrai57endiNwMFAwVnaWp%2BbmguLm0BBgZmaGN%2BYWlrewQHTQVgaGN5Zm9keAsABwMraGN8K2oue04ATQQrai57K2p%2FZgJMAAFla2J4b29reU5VbgVgaGN5Zm9keAsABwMra2V5ZmhjfgQDCARhbS55Zm0ue04ATQQrai57K2ouex8dAUhmb2B6Z2lqfgsCTVEIa2V5ZmhjfgQDCARhbS56YGhjeQMFBwduamR8K2hjfE4ATQQrai57K2oue04AHBlnJmN%2BZWtieAoFCAYrPw16YGhjeQMFBwduamR8K2tleQMCAAFhaWt7YW0ueQMHTQQrai57K2oue04ATQR6d2I3Zm9gegIDCQFuaC4uCGtleQMCAAFhaWt7YW0uegUCAAZmb2R4bmpkfE4CAAMrai57K2oue04ATQQran9mZyZjfgABAQdvb2t5Kz8NegUCAAZmb2R4bmpkfE4BBgZmaGN%2BYWlrewQHTQZmbS57K2oue04ATQQrai57endiNwMFAwVnaWp%2BbmguLm0BBgZmaGN%2BYWlrewQHTQVgaGN5Zm9keAsABwMraGN8K2oue04ATQQrai57K2p%2FZgJMAAFla2J4b29reU5VbgVgaGN5Zm9keAsABwMra2V5ZmhjfgQDCARhbS55Zm0ue04ATQQrai57K2ouex8dAUhmb2B6Z2lqfgsCTVEIa2V5ZmhjfgQDCARhbS56YGhjeQMFBwduamR8K2hjfE4ATQQrai57K2oue04AHBlnJmN%2BZWtieAoFCAYrPw16YGhjeQMFBwduamRzK2tleQMCAAFhaWt7YWIueQMHTQQrai57K2oue04ATQR6d2I3Zm9gegIDCQFuaC4uCGtleQMCAAFhaWt7YWIuegUCAA

In [ ]:
# # Liczba unikatowych użytkowników, którzy weszli zarówno w dodatek, jak i w oryginał
users_in_both = unique_users_additional_leaflets.intersection(unique_users_original_leaflet)

# Konwersja: liczba użytkowników, którzy weszli najpierw w dodatek, a następnie w oryginał (punkt 4), podzielona przez liczbę użytkowników w obu
conversion_rate = (len(unique_first_additional_then_original) / len(users_in_both) * 100) if len(users_in_both) != 0 else 0

# Display results
print("Data has been saved to 'combined_leaflet_data.csv'.")

print("1. Liczba unikatowych użytkowników, którzy weszli w dodatek, ale nie weszli w oryginał:")
print(len(users_only_in_additional_leaflets))

print("\n2. Liczba unikatowych użytkowników dodatku:")
print(len(unique_users_additional_leaflets))

print("\n3. Liczba unikatowych użytkowników oryginału:")
print(len(unique_users_original_leaflet))

print("\n4. Liczba unikatowych użytkowników, którzy weszli najpierw w dodatek, a następnie w oryginał:")
print(len(unique_first_additional_then_original))

print("\n5. Liczba unikatowych użytkowników, którzy weszli zarówno w dodatek, jak i w oryginał:")
print(len(users_in_both))

print(f"\n6. Konwersja (liczba użytkowników, którzy weszli najpierw w dodatek, a następnie w oryginał, podzielona przez liczbę użytkowników w obu): {conversion_rate:.2f}%")



Data has been saved to 'combined_leaflet_data.csv'.
1. Liczba unikatowych użytkowników, którzy weszli w dodatek, ale nie weszli w oryginał:
198179

2. Liczba unikatowych użytkowników dodatku:
302974

3. Liczba unikatowych użytkowników oryginału:
139765

4. Liczba unikatowych użytkowników, którzy weszli najpierw w dodatek, a następnie w oryginał:
14874

5. Liczba unikatowych użytkowników, którzy weszli zarówno w dodatek, jak i w oryginał:
104795

6. Konwersja (liczba użytkowników, którzy weszli najpierw w dodatek, a następnie w oryginał, podzielona przez liczbę użytkowników w obu): 14.19%


Obliczenie w przypadku większej ilości dodatków do jednego oryginału

In [ ]:
import pandas as pd
import requests
from pandas import json_normalize

# Function to fetch leaflet data from API
def fetch_leaflet_data(leaflet_id, start, end, headers):
    final_df = pd.DataFrame()
    data = {
        "event_name": "leaflet_enter",
        "event_properties": [{"name": "leaflet_id", "operator": "equals", "value": leaflet_id}],
        "from": start,
        "to": end
    }

    try:
        response = requests.post('https://eu1.api.clevertap.com/1/events.json', headers=headers, json=data)
        response.raise_for_status()  # Ensure we raise an exception for bad status codes
        response_json = response.json()
        req_id = response_json.get('cursor')

        while req_id:
            print(f"Fetching data for cursor: {req_id}")  # Logging the cursor to track progress
            params = (('cursor', req_id),)
            response1 = requests.get(f'https://eu1.api.clevertap.com/1/events.json?cursor={req_id}', headers=headers, params=params)
            response1.raise_for_status()  # Ensure we raise an exception for bad status codes
            response_data = response1.json()
            records = response_data.get('records', [])
            response_df = json_normalize(records)

            if 'ts' in response_df.columns:
                response_df = response_df[['ts', 'event_props.leaflet_id', 'profile.identity']]
                response_df.rename(columns={'event_props.leaflet_id': 'leaflet_id', 'profile.identity': 'user_id'}, inplace=True)
                response_df['ts'] = response_df['ts'].astype(str).apply(lambda x: x[:8])
                final_df = pd.concat([final_df, response_df], ignore_index=True)

            req_id = response_data.get('next_cursor')
            if not req_id:  # If no next cursor, break the loop
                break

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

    return final_df

# API headers
headers = {
    'X-CleverTap-Account-Id': 'R95-8KW-ZR5Z',
    'X-CleverTap-Passcode': '7dd63a92-b9c9-40c3-a004-330f9739e431',
    'Content-Type': 'application/json',
}

# Date range for the data
start = '2024-06-18'
end = '2024-06-24'

# Leaflet IDs
additional_leaflet_ids = ['368055', '368358']  # List of additional leaflet IDs
original_leaflet_id = '365597'  # Original leaflet ID

# Convert dates to integers
start = int(start.replace('-', ''))
end = int(end.replace('-', ''))

# Fetch data for additional leaflets
additional_leaflet_data = pd.DataFrame()
for leaflet_id in additional_leaflet_ids:
    additional_leaflet_data = pd.concat([additional_leaflet_data, fetch_leaflet_data(leaflet_id, start, end, headers)], ignore_index=True)

# Fetch data for the original leaflet
original_leaflet_data = fetch_leaflet_data(original_leaflet_id, start, end, headers)

# Combine the data into a single DataFrame
conversion = pd.concat([additional_leaflet_data, original_leaflet_data], ignore_index=True)

# Save the combined data to a CSV file
#conversion.to_csv('combined_leaflet_data.csv', index=False)

# Unique users for each leaflet
unique_users_additional_leaflets = set(additional_leaflet_data['user_id'])
unique_users_original_leaflet = set(original_leaflet_data['user_id'])

# Users only in additional leaflets
users_only_in_additional_leaflets = unique_users_additional_leaflets - unique_users_original_leaflet

# Users who entered additional leaflets first, then original leaflet
additional_leaflet_data_sorted = additional_leaflet_data.sort_values(by='ts')
original_leaflet_data_sorted = original_leaflet_data.sort_values(by='ts')

# Create a DataFrame with the first entry of each user in each leaflet
first_entry_additional_leaflets = additional_leaflet_data_sorted.groupby('user_id').first().reset_index()
first_entry_original_leaflet = original_leaflet_data_sorted.groupby('user_id').first().reset_index()

# Merge the DataFrames to find users who entered both leaflets
merged_entries = pd.merge(first_entry_additional_leaflets, first_entry_original_leaflet, on='user_id', suffixes=('_additional', '_original'))

# Find users who entered additional leaflets first, then original leaflet
unique_first_additional_then_original = merged_entries[merged_entries['ts_additional'] < merged_entries['ts_original']]['user_id'].unique()

# Display results
print("Data has been saved to 'combined_leaflet_data.csv'.")

print("1. Liczba unikatowych użytkowników, którzy weszli w dodatek, ale nie weszli w oryginał:")
print(len(users_only_in_additional_leaflets))

print("\n2. Liczba unikatowych użytkowników dodatku:")
print(len(unique_users_additional_leaflets))

print("\n3. Liczba unikatowych użytkowników oryginału:")
print(len(unique_users_original_leaflet))

print("\n4. Liczba unikatowych użytkowników, którzy weszli najpierw w dodatek, a następnie w oryginał:")
print(len(unique_first_additional_then_original))


Fetching data for cursor: ZyZjfgABAQdvb2t5Kz8NegUCAAZmbGF5Zmhhe04BBgZmaGN9ZGhjeQEATQZmbS57K2oue04ATQQrai57endiNwMFAwVnaWp%2BbmguLm0BBgZmaGN9ZGhjeQEATQVgaGN5ZmxheQMCAgQraGN8K2oue04ATQQrai57K2p%2FZgJMAAFla2J4b29reU5VbgVgaGN5ZmxheQMCAgQra2V5ZmhjfQECAAZkai55Zm0ue04ATQQrai57K2ouex8dAUhmb2B6Z2lqfgsCTVEIa2V5ZmhjfQECAAZkai56YGhjeQMGAgZmaGF7K2hjfE4ATQQrai57K2oue04AHBlnJmN%2BZWtieAoFCAYrPw16YGhjeQMGAgZmaGF7K2tleQMCAAJkaGN5ZGoueQMHTQQrai57K2oue04ATQR6d2I3Zm9gegIDCQFuaC4uCGtleQMCAAJkaGN5ZGouegUCAAZmbGF5Zmhhe04CAAMrai57K2oue04ATQQran9mZyZjfgABAQdvb2t5Kz8NegUCAAZmbGF5Zmhhe04BBgZmaGN9ZGhjeQEATQZmbS57K2oue04ATQQrai57endiNwMFAwVnaWp%2BbmguLm0BBgZmaGN9ZGhjeQEATQVgaGN5ZmxheQMCAgQraGN8K2oue04ATQQrai57K2p%2FZgJMAAFla2J4b29reU5VbgVgaGN5ZmxheQMCAgQra2V5ZmhjfQECAAZkai55Zm0ue04ATQQrai57K2ouex8dAUhmb2B6Z2lqfgsCTVEIa2V5ZmhjfQECAAZkai56YGhjeQMGAgZmaGF7K2hjfE4ATQQrai57K2oue04AHBlnJmN%2BZWtieAoFCAYrPw16YGhjeQMGAgZmaGF7K2tleQMCAAJkaGN5ZGoueQMHTQQrai57K2oue04ATQR6d2I3Zm9gegIDCQFuaC4uCGtleQMCAAJkaGN5ZGouegUCAAZmbGF5Zm

<ipython-input-10-fa9e0d17db45>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  response_df.rename(columns={'event_props.leaflet_id': 'leaflet_id', 'profile.identity': 'user_id'}, inplace=True)
<ipython-input-10-fa9e0d17db45>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  response_df['ts'] = response_df['ts'].astype(str).apply(lambda x: x[:8])


Fetching data for cursor: ZyZjfgABAQdvb2t5Kz8NegUCAAZmbGF5Zmhhe04BBgZmaGN9ZGhjeQEATQZmbS57K2hieQYABwVvJn96K25mfgIITQZnaGZ7YWtqNx8BTQFuaGdzendiNwMFAwVnaWp%2BbmguLm0BBgZmaGN9ZGhjeQEATQVgaGN5ZmxheQMCAgQraGN8K2oueQICBQRha2o3emsufwYGBwUraGJ5Y2pkegpMHAUrb2t%2BYmh%2FZgJMAAFla2J4b29reU5VbgVgaGN5ZmxheQMCAgQra2V5ZmhjfQECAAZkai55Zm0ue04CAQZjamR6byZ%2Fek4EBQNhbS55Z2hmewQBCUh6ay5%2Bbm1keR8dAUhmb2B6Z2lqfgsCTVEIa2V5ZmhjfQECAAZkai56YGhjeQMGAgZmaGF7K2hjfE4ATQZnaGZ7YWtqNx8BTQBjb2Z5K2hieQYABwVvJn96K29reAAJHBlnJmN%2BZWtieAoFCAYrPw16YGhjeQMGAgZmaGF7K2tleQMCAAJkaGN5ZGoueQMHTQQraGJ5Y2pkegpMHAUrbmZ9YGkueQICBQRha2o3emsufgsEBAZ6d2I3Zm9gegIDCQFuaC4uCGtleQMCAAJkaGN5ZGouegUCAAZmbGF5Zmhhe04CAAMrai55Z2hmewQBCUh6ay5%2FY29heU4CAQZjamR6byZ%2Fek4FCAFlaH9mZyZjfgABAQdvb2t5Kz8NegUCAAZmbGF5Zmhhe04BBgZmaGN9ZGhjeQEATQZmbS57K2hieQYABwVvJn96K25meAcGTQZnaGZ7YWtqNx8BTQFua2p7endiNwMFAwVnaWp%2BbmguLm0BBgZmaGN9ZGhjeQEATQVgaGN5ZmxheQMCAgQraGN8K2oueQICBQRha2o3emsufwYCCQIraGJ5Y2pkegpMHAUrb2t5ZGJ%2FZgJMAAFla2J4b29reU5VbgVgaGN5ZmxheQMCAg